In [3]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

DATA_PATH = os.getenv('TEACHER_DIR', os.getcwd()) + '/JHL_data'

### Let's try to create a differt type of model: A Convolutional Neural Network ('CNN')

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # conv: 1x28x28 -> 32x26x26
        # relu
        # conv: 32x26x26 -> 64x24x24
        # relu
        # max_pool: 64x24x24 -> 64x12x12
        # flatten: 64x12x12 -> 9216
        # linear: 9216 -> 128
        # relu
        # linear: 128 -> 10
        # log_softmax
        
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.relu2 = nn.ReLU()
        self.max_pool = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
#         self.dropout1 = nn.Dropout(0.25)  # optional
#         self.dropout2 = nn.Dropout(0.5)  # optional
        self.fc1 = nn.Linear(9216, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)
        

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.max_pool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

### Insert your train loop that you created for the MLP here

In [5]:
def train(model, device, train_loader, optimizer, epoch, log_interval=10):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            pred = output.argmax(dim=1, keepdim=True) 
            correct = pred.eq(target.view_as(pred)).sum().item()
            
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(),
                correct/len(data)
            ))

### Insert your test loop that you created for the MLP here

In [6]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100 * correct / len(test_loader.dataset)))


### Training hyperparameters and PyTorch boilerplate

In [7]:
use_cuda = torch.cuda.is_available()
print(f"CUDA is {'' if use_cuda else 'not '}available")
device = torch.device("cuda" if use_cuda else "cpu")

CUDA is not available


In [9]:
BATCH_SIZE = 600
EPOCHS = 15
LEARNING_RATE = 0.01
LOG_INTERVAL = 10

model = CNN().to(device)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

transform = transforms.Compose([
    transforms.ToTensor(), # Creates the PyTorch tensors from the PIL images, and normalizes them to the [0, 1] interval 
    transforms.Normalize((0.1307,), (0.3081,))  # Normalizes the data to 0 mean and 1 standard deviation
])

train_loader, test_loader = (
    torch.utils.data.DataLoader(
        datasets.MNIST(DATA_PATH, train=train, transform=transform, download=True),
        batch_size=BATCH_SIZE,
        pin_memory=True
    )
    for train in (True, False)
)

for epoch in range(EPOCHS):
    train(model, device, train_loader, optimizer, epoch, LOG_INTERVAL)
    test(model, device, test_loader)



3.4%

31.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100.0%


Extracting /home/robogast/Code/UvA_ML_2022/notebooks/JHL_data/MNIST/raw/train-images-idx3-ubyte.gz to /home/robogast/Code/UvA_ML_2022/notebooks/JHL_data/MNIST/raw


24.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Train Epoch: 0 [6000/60000 (10%)]	Loss: 2.192132	Accuracy: 0.48
Train Epoch: 0 [12000/60000 (20%)]	Loss: 2.045283	Accuracy: 0.60
Train Epoch: 0 [18000/60000 (30%)]	Loss: 1.784880	Accuracy: 0.66
Train Epoch: 0 [24000/60000 (40%)]	Loss: 1.371057	Accuracy: 0.77


KeyboardInterrupt: 

### Do you notice something when comparing the training and validation losses?
### Is there something you can do about it?

# CIFAR10

MNIST is the easiest widely-used dataset as a computer vision toy-problem.
One step above MNIST is CIFAR10, which contains images from 10 classes of objects:

![image.png](attachment:image.png)



In [22]:
class CIFAR10CNN(nn.Module):
    def __init__(self):
        super().__init__()

        # 4 convolution layers, each followed by _batch-normalization_
        # (try to find out what this is and why!), and a non-linearity
        # maxpooling after the activations of the 2nd, 3rd, and 4th conv layers
        # 2 dense layers for classification
        # log_softmax
        #
        # As for the number of channels of each layers, try to experiment!

        self.layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=8),
            nn.ReLU(),
            nn.Conv2d(in_channels=8, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),
            nn.Linear(in_features=2048, out_features=2048),
            nn.ReLU(),
            nn.Linear(in_features=2048, out_features=10),
            nn.LogSoftmax()
        )



    def forward(self, x):
        return self.layers(x)


In [26]:
# We can Re-use our train and test loops

BATCH_SIZE = 128
EPOCHS = 15
LEARNING_RATE = 0.001
LOG_INTERVAL = 10

model = CIFAR10CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE) # optim.<OPTIMIZER_FLAVOUR>(model.parameters(), lr=LEARNING_RATE)


transform=transforms.Compose([
    transforms.ToTensor(),
    # Normalize the data to 0 mean and 1 standard deviation, now for all channels of RGB
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])


train_loader, test_loader = (
    torch.utils.data.DataLoader(
        datasets.CIFAR10(DATA_PATH, train=train, transform=transform, download=True),
        batch_size=BATCH_SIZE,
        pin_memory=True,
        shuffle=True
    )
    for train in (True, False)
)

for epoch in range(EPOCHS):
    train(model, device, train_loader, optimizer, epoch, LOG_INTERVAL)
    test(model, device, test_loader)

Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 0 [0/50000 (0%)]	Loss: 2.349461	Accuracy: 0.11
Train Epoch: 0 [1280/50000 (3%)]	Loss: 2.267515	Accuracy: 0.25
Train Epoch: 0 [2560/50000 (5%)]	Loss: 2.083044	Accuracy: 0.25
Train Epoch: 0 [3840/50000 (8%)]	Loss: 1.839039	Accuracy: 0.30
Train Epoch: 0 [5120/50000 (10%)]	Loss: 1.778747	Accuracy: 0.33
Train Epoch: 0 [6400/50000 (13%)]	Loss: 1.719884	Accuracy: 0.38
Train Epoch: 0 [7680/50000 (15%)]	Loss: 1.539471	Accuracy: 0.42
Train Epoch: 0 [8960/50000 (18%)]	Loss: 1.634732	Accuracy: 0.41
Train Epoch: 0 [10240/50000 (20%)]	Loss: 1.593994	Accuracy: 0.38
Train Epoch: 0 [11520/50000 (23%)]	Loss: 1.541728	Accuracy: 0.45
Train Epoch: 0 [12800/50000 (26%)]	Loss: 1.449866	Accuracy: 0.45
Train Epoch: 0 [14080/50000 (28%)]	Loss: 1.405400	Accuracy: 0.51
Train Epoch: 0 [15360/50000 (31%)]	Loss: 1.334575	Accuracy: 0.52
Train Epoch: 0 [16640/50000 (33%)]	Loss: 1.465970	Accuracy: 0.49
Train Epoch: 0 [17920/50000 (

KeyboardInterrupt: 